# Chapter 11 toxと継続的インテグレーション

チームで作業を行う場合、各開発者はコードに加えた変更を共有リポジトリに定期的にマージを行うことが多い。このようなことを「継続的インテグレーション（CI）」と呼ぶ。CI は共同作業のときだけでなく、一人で作業を行う場合にも非常に役立つ手法である。

CI に使われるほとんどのツール（例えば Github Actions 等）はサーバー上で実行される。本章で取り扱う「tox」は CI ツールとよく似た自動化ツールだが、ローカルで実行することも、他の CI ツールと組み合わせてサーバー上で実行することもできる。

Cards アプリケーションをローカルでテストするために tox を設定する方法や、Github Actions を使い Github でテストを行う方法について見ていこう。


## 11.1 継続的インテグレーション (CI) とは

統合 ＝ インテグレーション

かつて CI が使われていない頃は、開発者が異なるコードブランチで機能を追加し、コードの準備ができたら（週1回の頻度、等で）マージ（統合）をしていた。

このような状況では競合が多々発生し、コーディングが快適な環境とは言えない。そこで誕生したのが CI である。

CI ツールとは、ビルドとテスト実行をすべて自動的に行う。通常、マージリクエストによって開始され、ビルド段階とテスト段階が自動化されるため、開発者は頻繁に（継続的に）統合を行うことができる。この仕組みは一人での開発でも有用である。

## 11.2 速習：tox

tox は完全なテストスイート※を複数の環境で実行できるようにするコマンドラインツール（※特定のテストサイクルで実行されるテストケースやテスト手順のセット）。

tox は厳密には CI システムではないが、よく似ており、ローカルで実行することができる。

tox は、テスト対象の「パッケージの setup.py」または「pyproject.toml に格納されているプロジェクト情報」を使って、そのパッケージのインストール可能なディストリビューション（ソフトウェア群）を作成する。

そして、tox.ini で環境のリストを調べた後、環境ごとに次の作業を行う：

1. 依存パッケージをインストール（pip install）
2. パッケージをビルド
3. パッケージをインストール（pip install）
4. テストを実行


## 11.3 tox を使うための準備

10章まででは、cards.proj のコードを1つのディレクトリに格納し、テストを各章のディレクトリに格納していた。

ここでは、それらを一つのプロジェクトにまとめ、tox.ini ファイルを追加する。

```plaintext
cards.proj
├── LICENSE
├── README.md
├── Pyproject.toml
├── pytest.ini
├── src
│   └── cards
│       └── ...
├── tests
│   ├── api
│   │   └── ...
│   └── cli
│       └── ...
└── tox.ini
```

ch11/cards.proj/tox.ini
```plaintext
[tox]
envlist = py310
isolated_build = True

[testenv]
deps =
    pytest
    faker
commands = pytest

```

[tox]
- envlist = py310 : tox に Python 3.10 を使ってテストを実行させるための省略表記。
- isolated_build = True : pyproject.toml ファイルでビルド命令の設定を行うプロジェクトでは、この isolated_build = True という設定が必要（setuptools ライブラリを使って setup.py で設定を行う場合、省略可）

[testenv]
- deps = X : 「テストを行う際にツール X のインストールが必要である」ことを tox に伝える。pytest == 6.2.4 (>=等も可) のようにパージョンを指定することもできる。
- commands = Y: 「tox に各環境（envlist = ...）で コマンド Y を実行させる」


## 11.4 tox を実行する

tox を実行するには、tox を（仮想環境等に）インストールする。

```
$ pip install tox

```

今回のようなファイル構成であれば、単に tox を実行するだけで良い。

実行すると、すべてのテスト環境（envlist で指定した部分）のサマリー情報と、それらの結果を出力する。

```
$ cd <code/ch11/cards.proj へのパス>
$ tox
py310 create <code/ch11/cards-proj へのパス>/.tox/py310 
py310 install_deps: pytest, faker
```

## 11.5 Python の複数のバージョンでテストする

Python バージョン 3.7 ~ 3.10 でテストを実行するには下記のように書き換えれば良い。

skip_missing_interpreters = True はFalse(デフォルト) の場合、envlist に指定されているバージョンの中に、システムにインストールされていないものが一つでもある場合に、tox が失敗する。True の場合、tox は失敗せず、利用可能なバージョンでテストし、見つからないバージョンはスキップする。

※ 本来、下記のように tox.ini を名前を変えて複数用意することは無い

ch11/cards.proj/tox_multiple_pythons.ini
```plaintext
[tox]
envlist = py37, py38, py39, py310
isolated_build = True
skip_missing_interpreters = True

[testenv]
deps =
    pytest
    faker
commands = pytest

```

実行例

## 11.6 複数の tox 環境を同時に実行する

前節の例では、複数の環境を順番に実行していた。tox 実行時に -p フラグをつけることで、それらの環境を同時に実行することもできる。

## 11.7 カバレッジレポートを tox に追加する

テストの実行にカバレッジレポート（Chapter 9 参照）を追加したい場合、tox.ini に以下の変更を加えれば良い：
- deps の設定に pytest-cov を追加
  - tox テスト環境に pytest-cov プラグインがインストールされるようにする
  - pytest-cov を追加すると coverage などの依存パッケージもすべてインストールされる
- commands 呼び出しをpytest --cov=cards とする


リスト11-3: ch11/cards.proj/tox_coverage.ini
```plaintext
[testenv]
deps =
    pytest
    faker
    pytest-cov
commands = pytest --cov=cards
```

tox で coverage を使うときには、どのソースパスを同一と見なせばよいかを教えるために .coveragerc ファイルも設定するとよい。

リスト11-4: ch11/cards.proj/.coveragerc
```plaintext
[paths]
source=
    src
    .tox/*/site-package
```

tox を実行すると仮想環境を .tox ディレクトリに作成する (例: .tox/py310)。Cards プロジェクトのソースコードは、tox を実行する前は src/cards ディレクトリにある。しかし、cards パッケージを tox がインストールすると、Cards プロジェクトのソースコードは .tox のどこかに埋もれた site-packages/cards ディレクトリに存在するようになる。たとえば Python 3.10 では、.tox/py310/lib/python3.10/site-packages/cards に現れる。

coverage の source 設定のリストに src と.tox/*/site-packages を追加するのは、先のコードから次の出力を得るための省略表記である。

上例の -e py310 は特定の環境を選択するオプション

## 11.8 カバレッジのベースラインを指定する

coverage を tox から実行する際、カバレッジのベースライン (パーセンテージ) を設定して、ベースラインを満たさないものが1つでもあれば警告するようにしておくのも効果的である。--cov-fail-under フラグを使うことで実行できる。

commands の他のフラグについては下記の通り
- tox.ini ファイルで pytest の呼び出しに --cov=tests を追加して、すべてのテストが実行される
- tox のコマンドラインに -e py310 を追加し、環境を python 3.10 一つに絞り込む

11-5: ch11/cards.proj/tox_coverage_min.ini
[testenv]
deps =
    pytest
    faker
    pytest-cov
commands = pytest --cov=cards --cov=tests --cov-fail-under=100


出力結果

## 11.9 tox から pytest にパラメータを渡す

commands の pytest コマンドに {posargs} を追加することで、pytest にパラメータを渡せるようになる。

リスト11-6:ch11/cards.proj/tox_posargs.ini
[testenv]
deps =
    pytest
    faker
    pytest-cov
commands=
    pytest cov=cards --cov=tests --cov-fail-under=100 {posargs}

pytest にパラメータを渡すには、tox のパラメータと pytest のパラメータの間に--を追加する。例として、キーワードフラグ -k を使って test_version テストを選択することにする。また、 --no-cov フラグを使って coverage を無効にする (テストを2つ実行するだけならカバレッジを計測しても意味がないためである)。


実行結果

## 11.10 Github Actions を使って tox を実行する

コードをコミットまたはマージする際、変更のたびに必ず tox を実行するように CIシステムを設定しておくと非常に便利です。

GitHub Actions は GutHub が提供しているクラウドベースの CI システムである。プロジェクトのリポジトリとして GitHub を使っているのであれば、Actions は CI として自然
な選択肢であろう。

Github リポジトリに Actions を追加するには、単にワークフローファイルである.ymlを、プロジェクトのトップレベルにある .github/workflows/ディレクトリ
に追加するだけです。例えば Cards プロジェクトの main.yml は以下のように記述している（Github Actions を一部 GA と略記している）：


```main.yml
name: CI    # Github Actions のユーザーインターフェースに表示される名前

on: [push, pull_request]    # 誰かが[]内をする際にテストが実行される

jobs:
  build:

    runs-on: ubuntu-latest    # テストをどのOSで実行するか指定
    strategy:
      matrix:
        python: ["3.7", "3.8", "3.9", "3.10"]    # Python のバージョン指定

    steps:    # ステップのリスト。各 name は何を指定しても、省略しても良い
      - uses: actions/checkout@v2    # リポジトリをチェックアウトしてワークフローの残り部分からアクセスできるようにする GA のツール
      - name: Setup Python
        uses: actions/setup-python@v2    # Python を設定してビルド環境にインストールする GA のツール
        with:
          python-version: ${{ matrix.python }}
      - name: Install Tox and any other packages
        run: pip install tox    # tox をインストール
      - name: Run Tox
        run: tox -e py    # toxを実行。-e py により tox.ini で指定された正しいバージョンの Python が正しく選択される
```

.yml ファイルを Github リポジトリにプッシュすると、ファイルの内容が自動で実行されるようになる。

これまでの実行結果を確認するには[Actions]タブを選択する。

トップレベルの name 設定である main が一番上に表示されており、各ステップの名前が右側に表示される。